In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms,models
import matplotlib.pyplot as plt
import torch.nn.functional as F
from tqdm import tqdm

import numpy as np
import csv

In [2]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 64x64
    transforms.ToTensor(),  # Convert images to tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize the images
])

train_dir = '/kaggle/input/iith-dl-contest-2024/train/train'
train_data  = datasets.ImageFolder(train_dir, transform = transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)

In [3]:
device = 'cuda'

In [4]:
model = models.convnext_tiny(weights=None)

In [5]:
#model.classifier[1] = nn.Sequential(nn.ReLU(nn.Linear(1408,470)),nn.ReLU(nn.Linear(470,156)),nn.Linear(156,50))

model.classifier[2] = nn.Linear(768,50)

In [6]:
model.to(device)

ConvNeXt(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      (1): LayerNorm2d((96,), eps=1e-06, elementwise_affine=True)
    )
    (1): Sequential(
      (0): CNBlock(
        (block): Sequential(
          (0): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
          (1): Permute()
          (2): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
          (3): Linear(in_features=96, out_features=384, bias=True)
          (4): GELU(approximate='none')
          (5): Linear(in_features=384, out_features=96, bias=True)
          (6): Permute()
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): CNBlock(
        (block): Sequential(
          (0): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
          (1): Permute()
          (2): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
          (3): Linear(in_features=

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

total_steps = len(train_loader)

for epoch in range(16):
    
    Loss = 0

    for i , (images, label) in tqdm(enumerate(train_loader)):

        images = images.to(device)
        label = label.to(device)

        outputs = model(images)
        loss = criterion(outputs, label)
        with torch.no_grad():
            Loss += loss 
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(Loss)
    if epoch%2==0:
        torch.save(model.state_dict(), 'Convnext_tiny_model_weights{}.pth'.format(epoch+1))

1016it [25:47,  1.52s/it]


tensor(3368.4377, device='cuda:0')


1016it [20:51,  1.23s/it]


tensor(2799.2039, device='cuda:0')


1016it [20:41,  1.22s/it]


...

tensor(295.7393, device='cuda:0')


1016it [21:17,  1.26s/it]


tensor(251.4720, device='cuda:0')


1016it [21:06,  1.25s/it]

tensor(220.0435, device='cuda:0')


In [8]:
classes = train_data.class_to_idx
idx_to_class = {idx: class_name for class_name, idx in classes.items()}

test_dir = '/kaggle/input/iith-dl-contest-2024/test'
test_data  = datasets.ImageFolder(test_dir, transform = transform)

test_loader = torch.utils.data.DataLoader(test_data, batch_size = 64, shuffle = False)

list_of_outs = []



In [9]:
list_of_outs = []

for i, (images, labels) in tqdm(enumerate(test_loader)):
    
    images = images.to(device)
    
    outputs = model(images)
    _, outputs = torch.max(outputs.data, 1)

    list_of_outs.append(outputs)
        
outs = torch.cat(list_of_outs)

600it [05:01,  1.99it/s]


In [10]:
right = outs.cpu().numpy()
right = np.array([idx_to_class[_] for _ in right], dtype = object)
right.shape



left = []
for i in range(38366):
    s = "{}.JPEG".format(i)
    left.append(s)
left = np.sort(left)

table = np.transpose(np.array([left, right], dtype = object))

print(table)

import csv

with open('submission_Convnext.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['ID', 'Category'])
    writer.writerows(table)

[['0.JPEG' 'n02808440']
 ['1.JPEG' 'n02480495']
 ['10.JPEG' 'n02190166']
 ...
 ['9997.JPEG' 'n02480495']
 ['9998.JPEG' 'n02480495']
 ['9999.JPEG' 'n01882714']]
